In [ ]:
#!pip3 install selenium
#!pip3 install tqdm
#!pip3 install ipywidgets
#!jupyter nbextension enable --py widgetsnbextension

In [1]:
import pandas as pd
from time import sleep
from tqdm.notebook import tqdm, trange

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

## Bitso Historical Live Scraping

In [2]:
coins=['btc','eth','xrp','ltc','bch','tusd','mana','gnt','bat','dai']
URLs={}
for i in coins:
    URLs.update({i:'https://bitso.com/trade/market/'+i+'/mxn'})

In [3]:
database=[]

In [4]:
def readscreen(test=False, verbose=False, database=database):
    try:
        # Getting candels tags
        tagsPath = driver.find_elements_by_tag_name('g')
        candels=[]
        for i in range(len(tagsPath)):
            if tagsPath[i].get_attribute("aria-label") != None and tagsPath[i].get_attribute("aria-label").find('Price')==0:
                #print(tagsPath[i].get_attribute("transform"), tagsPath[i].get_attribute("aria-label"))
                candels.append(tagsPath[i])
        print(len(candels), 'Candels')

        #Checking every candel by moving arround
        
        for i in trange(len(candels)):
            try:
                action=ActionChains(driver)
                action.move_to_element(candels[i])
                action.perform()
                #print(i, candels[i].get_attribute('aria-label'))
                
                #Getting info of every candel

                voltag=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div/div[1]/div/div/div[2]/div[2]/div/div/div/div')
                volume=voltag.text

                boxtags = driver.find_elements_by_tag_name('b')
                i_open=boxtags[0].text
                i_low=boxtags[1].text
                i_high=boxtags[2].text
                i_close=boxtags[3].text

                datetag=driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div/div[1]/div/div/div[2]/div[1]/div/div/div[2]/div')
                date=datetag.text

                day=[date,i_open,i_low,i_high,i_close,volume]
                
                if test == False:
                    database.append(day)

                if verbose == True:
                    print(day)
            except:
                print('Candel',i,'Fails')
    except:
            print('ERROR: Mayor Fail - Check the driver is initialized correctly')

In [5]:
def getScrollerPosition(si,sf):
    xi,yi=int(si[si.find('(')+1:si.find(',')]),int(si[si.find(',')+1:si.find(')')])
    xf,yf=int(sf[sf.find('(')+1:sf.find(',')]),int(sf[sf.find(',')+1:sf.find(')')])
    return xi,yi,xf,yf

In [13]:
def getScrollers():
    #Getting the tags of the  scrollers for time periods
    loaded=False
    while loaded == False:
        try:
            scrollers = driver.find_elements_by_tag_name('g') #The scrollers are over the tag 'g'
            scroll=[]

            for i in range(len(scrollers)):
                role = scrollers[i].get_attribute("role") # Scrollers has the exlusive attribute 'role'
                if role != None:
                    #print(scrollers[i].get_attribute("role"), scrollers[i].get_attribute("transform"))
                    scroll.append(scrollers[i])
            loaded=True
            if len(scroll) == 2:
                print('Scroller elements load successfully')
            else:
                print('Something went wrong with the scrollers')
        except:
            print('Loading...')

In [19]:
def scrap(zeroi=-17, zerof=38, max_tries=3, scroll=scroll):
    
    #Checking where are the scrollers
    si=scroll[0].get_attribute("transform")
    sf=scroll[1].get_attribute("transform")

    xi,yi,xf,yf=getScrollerPosition(si,sf)

    step=zerof-zeroi

    it=0
    tries=0
    
    # Movig the scrollers for time periods
    
    try:
        while 1:
            while (xi != zeroi+step*it or xf != zerof+step*it) and tries < max_tries:
                action=ActionChains(driver)
                action.drag_and_drop_by_offset(scroll[0],zeroi+step*it-xi,0)
                action.drag_and_drop_by_offset(scroll[1],zerof+step*it-xf,0)
                action.perform()

                si=scroll[0].get_attribute("transform")
                sf=scroll[1].get_attribute("transform")
                xi,yi,xf,yf=getScrollerPosition(si,sf)
                sleep(1)
                tries+=1
                print('Fitting the area to scan.','.'*tries)#xi,xf,'-',zeroi+step*it,zerof+step*it,'try:',tries)

            sleep(1)
            readscreen() ## Getting info function
            it+=1
            tries=0
            print('Iteracion: ',it)
    except:
        print('Espacio completado en ',it,'iteraciones')

In [20]:
def save(name,database=database):
    df=pd.DataFrame(database)
    df=df.drop_duplicates()
    df=df.reset_index()
    df.rename(columns={0:'date',1:'open',2:'low',3:'high',4:'close',5:'volume'},inplace=True)
    df=df.drop(columns='index')
    df['date']=pd.to_datetime(df['date'],format='%b %d, %Y')
    df.to_csv('./'+name+'-MXN.csv',index=False)

## Scrapper

In [32]:
for i in coins:
    database=[]
    driver = webdriver.Firefox()
    driver.get(URLs[i])
    driver.maximize_window()
    driver.execute_script("window.scrollTo(0, 200)") 
    scroll = getScrollers()
    scrap()
    save(i)
    print(i, ' Scrapped succesfully')
    driver.close()

Scroller elements load successfully


TypeError: 'NoneType' object is not subscriptable

## Other Sources

In [ ]:
import requests
from datetime import date

In [2]:
books={'Bitcoin':'btc_mxn',
       'Ethereum':'eth_mxn',
       'Ripple':'xrp_mxn',
       'LTC':'ltc_mxn',
       'Bitcoin Cash':'bch_mxn',
       'True Dollar':'tusd_mxn',
       'Mana':'mana_mxn',
       'Golem':'gnt_mxn',
       'BAT':'bat_mxn',
       'DAI':'dai_mxn'}

coins=['bitcoin','ethereum','xrp','litecoin','bitcoin-cash','trueusd','decentraland','golem-network-tokens',
       'basic-attention-token','multi-collateral-dai']
start='20120428'

end= date.today()
end= end.strftime("%Y%m%d")

url=[]
for coin in coins:
    url.append('https://coinmarketcap.com/currencies/'+coin+'/historical-data/?start='+start+'&end='+end)

In [199]:
url[0]

'https://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20120428&end=20200529'